## Nueral network
- 퍼셉트론, 다층 퍼셉트론(MLP, multilayer perceptron)
- Activation function, Gradient Descent, SGD
- Back propagation


## Sequential 
- 가장 간단한 모델
- 선형 파이프라인(스택)을 정의

#### Dense ?
- 각 레이어의 뉴런들이, 인접한 레이어의 모든 뉴런들과 빽빽히 연결되어 있는 구조

In [1]:
from keras.layers.core import Dense
from keras.models import Sequential

model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer="random_uniform"))

Using TensorFlow backend.


#### Weight 초기화 옵션들 ( https://keras.io/initializers/ )
- random_uniform, random_normal, zero 등등..
- 참고자료 ; uniform distribution vs. normal distribution  
   (https://www.quora.com/What-is-the-difference-between-normal-distribution-and-uniform-distribution)

#### Activation function ( https://keras.io/activations/ )
- sigmoid ; f(x) = 1 / (1 + e^(-x))
- ReLU ; f(x) = max(0, x)
- etc.

## MNIST 예제로 감잡기
- 6만개 학습 셋, 1만개 테스트 셋
- 각 클래스는 One hot encoding으로 표현

In [3]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

NB_EPOCH = 200
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
OPTIMIZER = SGD()
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 # training data 중 얼마나 validation set으로 쓸지

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 3s 0us/step


In [6]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [8]:
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED).astype('float32')
X_test = X_test.reshape(10000, RESHAPED).astype('float32')

# 정규화
X_train /= 255
X_test /= 255

In [13]:
model = Sequential()
model.add(Dense(NB_CLASSES, input_shape=(RESHAPED,)))
model.add(Activation('softmax'))

TypeError: softmax() got an unexpected keyword argument 'axis'